# Data preprocessing for Textmining

____

금일 처리해야하는 사항들 

* feed 내에서 나온 comment들을 모아야 합니다.
> output: [댓글, [대댓글,대댓글,대댓글]] 의 형태로 나타나게끔 묶어줍니다.

In [1]:
# 필요한 package 
import pandas as pd 
import ast #str to list
from tqdm import tqdm 
import tqdm
import numpy as np
import warnings 
warnings.filterwarnings("ignore",category=DeprecationWarning)
import re
import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier

In [2]:

def read_data():
    hyatt= pd.read_csv("hyatt_comment",index_col=[0]).reset_index()
    hilton=pd.read_csv("hilton_comment",index_col=[0]).reset_index()
    holiday=pd.read_csv("holiday_comment",index_col=[0]).reset_index()
    wyndham=pd.read_csv("wyndham_comment",index_col=[0]).reset_index()
    
    return (hilton,hyatt,holiday,wyndham)
hilton,hyatt,holiday,wyndham=read_data()
hilton.head()
test=hilton[:30]

In [3]:
# comment
def string_to_list(df,columns):
    for col in columns:
        df[col]=df[col].apply(lambda x: ast.literal_eval(x))
        
for hotel in (hilton,hyatt,holiday,wyndham):
    string_to_list(hotel,["comment","comment_like","comment_writer","Recomment"])

print(hilton['comment'][0][0])
hilton.head()# 애초에 list

❤️❤️❤️


,post_id,comment,comment_like,comment_writer,Recomment
0,1,"[❤️❤️❤️, 👏, 🤤❤️, OMG WHAT THIS IS AWESOME, I m...","[0, 0, 0, 2, 0, 0, 0, 0, 2, 3, 0, 1, 1, 1, 0, 0]","[3105070945, 17388782034, 1246744447, 11793739...","[[[@allpurple10 💙🍪, <Profile hilton (428491337..."
1,2,"[This is awesome!!!👏👏👏, Thank you!!, 👏🏻👏🏻👏🏻👏🏻👏...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0]","[3105070945, 4464995538, 1159581929, 291908643...","[[[@allpurple10 thank you!, <Profile hilton (4..."
2,3,"[Build with immortal love 👑, ❤️, 🧡🧡🧡, sooo cut...","[0, 0, 0, 0, 0]","[13811048459, 1139428551, 20534662365, 1997211...","[[], [], [], [], [[@erattray1 I saw that earli..."
3,4,[.\n.\n.\n.\n#staugustine #staugustinefl #stau...,[0],[4284913372],[[]]
4,5,[.\n.\n.\n.\n#boca #bocaraton #resortlife #exp...,"[0, 0, 0]","[4284913372, 534651664, 11230655630]","[[], [[@patriciatorressl Waldorf?, <Profile gi..."


comment 첫번째에 대한 답글 -> Recomment0에서 0에 있는 것들 
comment 두번째에 대한 답글 -> Recomment0에서 1에 있는 것들 
comment 세번째에 대한 답글 -> Recomment0에서 2에 있는 것들 

document=[comment,[recomment],]

In [4]:
def string_extract(x):
    # input : 3차 
    new_list=[]
    for i in x:
        if len(i)==0:
            new_list.append([])
            
        elif len(i)==1:
            new_list.append([i[0][0]])
        
        else : 
            string_lst=[]
            for j in range(len(i)-1):
                string_lst.append(i[j][0])
            new_list.append(string_lst)
    
    return new_list

In [5]:
for hotel in (hilton,hyatt,holiday,wyndham):
    hotel["new_recom"]=hotel["Recomment"].apply(lambda x: string_extract(x))
    hotel.drop(["Recomment"],axis=1,inplace=True)

hilton.head()

,post_id,comment,comment_like,comment_writer,new_recom
0,1,"[❤️❤️❤️, 👏, 🤤❤️, OMG WHAT THIS IS AWESOME, I m...","[0, 0, 0, 2, 0, 0, 0, 0, 2, 3, 0, 1, 1, 1, 0, 0]","[3105070945, 17388782034, 1246744447, 11793739...","[[@allpurple10 💙🍪], [@hamptonrockyhill we're p..."
1,2,"[This is awesome!!!👏👏👏, Thank you!!, 👏🏻👏🏻👏🏻👏🏻👏...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0]","[3105070945, 4464995538, 1159581929, 291908643...","[[@allpurple10 thank you!], [], [], [], [], []..."
2,3,"[Build with immortal love 👑, ❤️, 🧡🧡🧡, sooo cut...","[0, 0, 0, 0, 0]","[13811048459, 1139428551, 20534662365, 1997211...","[[], [], [], [], [@erattray1 I saw that earlie..."
3,4,[.\n.\n.\n.\n#staugustine #staugustinefl #stau...,[0],[4284913372],[[]]
4,5,[.\n.\n.\n.\n#boca #bocaraton #resortlife #exp...,"[0, 0, 0]","[4284913372, 534651664, 11230655630]","[[], [@patriciatorressl Waldorf?, @giles.ynwa ..."


new_recomment에 대한 정리는 끝    
-> 이제 comment와 new_recomment를 묶어보자. numpy가 아니기에 지금 굳이 묶을 필요는 x) ->묶으려면 아예 위계가 없게 묶는게 나은데 그건 추후에 하겠습니다.

## 가장 simple한 nltk 사용

우선 가장 먼저해야 하는 것은 text_cleaning 이지만 social 데이터의 경우에는 바로 text_cleaning을 하지 않는 것이 좋다. 이 연구에서는 그래서 text_cleaning을 하지 않고 진행하도록 하겠습니다. 우선 영어 코퍼스의 토큰화를 위한 도구를 NLTK를 통해서 진행하겠습니다.

In [205]:
# test=hilton[:30]
test.head()

,post_id,comment,comment_like,comment_writer,new_recom
0,1,"[❤️❤️❤️, 👏, 🤤❤️, OMG WHAT THIS IS AWESOME, I m...","[0, 0, 0, 2, 0, 0, 0, 0, 2, 3, 0, 1, 1, 1, 0, 0]","[3105070945, 17388782034, 1246744447, 11793739...","[[@allpurple10 💙🍪], [@hamptonrockyhill we're p..."
1,2,"[This is awesome!!!👏👏👏, Thank you!!, 👏🏻👏🏻👏🏻👏🏻👏...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0]","[3105070945, 4464995538, 1159581929, 291908643...","[[@allpurple10 thank you!], [], [], [], [], []..."
2,3,"[Build with immortal love 👑, ❤️, 🧡🧡🧡, sooo cut...","[0, 0, 0, 0, 0]","[13811048459, 1139428551, 20534662365, 1997211...","[[], [], [], [], [@erattray1 I saw that earlie..."
3,4,[.\n.\n.\n.\n#staugustine #staugustinefl #stau...,[0],[4284913372],[[]]
4,5,[.\n.\n.\n.\n#boca #bocaraton #resortlife #exp...,"[0, 0, 0]","[4284913372, 534651664, 11230655630]","[[], [@patriciatorressl Waldorf?, @giles.ynwa ..."


In [6]:
from nltk.tokenize import word_tokenize
import nltk 
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sunkeun_jo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
# comment에 대한 tokenize
def cleaned_text(text):
    # 누구를 tag한 태그 표시는 지우기
    #전의 text
    prev_text=text
    while True:
        text=re.sub("@[\d\w]+","",text)
        
        if prev_text!=text:
            prev_text=text
        else:
            break
            
    return text

def text_tokenize(text):
    return [word_tokenize(cleaned_text(sentence)) for sentence in text]

for hotel in (hilton,hyatt,holiday,wyndham):
    hotel["comment_token"]=hotel["comment"].apply(lambda x: text_tokenize(x))
    
hilton.head()

# comment에 대한 토큰화 작업 끝 

,post_id,comment,comment_like,comment_writer,new_recom,comment_token
0,1,"[❤️❤️❤️, 👏, 🤤❤️, OMG WHAT THIS IS AWESOME, I m...","[0, 0, 0, 2, 0, 0, 0, 0, 2, 3, 0, 1, 1, 1, 0, 0]","[3105070945, 17388782034, 1246744447, 11793739...","[[@allpurple10 💙🍪], [@hamptonrockyhill we're p...","[[❤️❤️❤️], [👏], [🤤❤️], [OMG, WHAT, THIS, IS, A..."
1,2,"[This is awesome!!!👏👏👏, Thank you!!, 👏🏻👏🏻👏🏻👏🏻👏...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0]","[3105070945, 4464995538, 1159581929, 291908643...","[[@allpurple10 thank you!], [], [], [], [], []...","[[This, is, awesome, !, !, !, 👏👏👏], [Thank, yo..."
2,3,"[Build with immortal love 👑, ❤️, 🧡🧡🧡, sooo cut...","[0, 0, 0, 0, 0]","[13811048459, 1139428551, 20534662365, 1997211...","[[], [], [], [], [@erattray1 I saw that earlie...","[[Build, with, immortal, love, 👑], [❤️], [🧡🧡🧡]..."
3,4,[.\n.\n.\n.\n#staugustine #staugustinefl #stau...,[0],[4284913372],[[]],"[[., ., ., ., #, staugustine, #, staugustinefl..."
4,5,[.\n.\n.\n.\n#boca #bocaraton #resortlife #exp...,"[0, 0, 0]","[4284913372, 534651664, 11230655630]","[[], [@patriciatorressl Waldorf?, @giles.ynwa ...","[[., ., ., ., #, boca, #, bocaraton, #, resort..."


comment와 recomment들을 하나로 묶어서 한 comment의 집단이 어떤 sentiment를 가지고 있는지 파악한다.

In [51]:
for hotel in (hilton,hyatt,holiday,wyndham):
    for i,rows in hotel.iterrows():# rows index 처음 row 를 한번에 부를 수 있습니다.
        com= np.array(rows["comment"]).reshape(-1,1)
        new_com=[]
        for j in rows["new_recom"]:
            sentence=""

            for x in j:
                sentence= sentence+" "+x
            new_com.append(sentence)

        new_com=np.array(new_com).reshape(-1,1)
        value= np.hstack([com,new_com])

        final_list=[]
        for word in value:
            sentence= word[0]+" "+word[1]

            final_list.append(sentence)

        hotel.loc[i,"final"]=str(final_list)

hilton.head()

,post_id,comment,comment_like,comment_writer,new_recom,comment_token,final
0,1,"[❤️❤️❤️, 👏, 🤤❤️, OMG WHAT THIS IS AWESOME, I m...","[0, 0, 0, 2, 0, 0, 0, 0, 2, 3, 0, 1, 1, 1, 0, 0]","[3105070945, 17388782034, 1246744447, 11793739...","[[@allpurple10 💙🍪], [@hamptonrockyhill we're p...","[[❤️❤️❤️], [👏], [🤤❤️], [OMG, WHAT, THIS, IS, A...","['❤️❤️❤️ @allpurple10 💙🍪', ""👏 @hamptonrockyh..."
1,2,"[This is awesome!!!👏👏👏, Thank you!!, 👏🏻👏🏻👏🏻👏🏻👏...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0]","[3105070945, 4464995538, 1159581929, 291908643...","[[@allpurple10 thank you!], [], [], [], [], []...","[[This, is, awesome, !, !, !, 👏👏👏], [Thank, yo...",['This is awesome!!!👏👏👏 @allpurple10 thank yo...
2,3,"[Build with immortal love 👑, ❤️, 🧡🧡🧡, sooo cut...","[0, 0, 0, 0, 0]","[13811048459, 1139428551, 20534662365, 1997211...","[[], [], [], [], [@erattray1 I saw that earlie...","[[Build, with, immortal, love, 👑], [❤️], [🧡🧡🧡]...","['Build with immortal love 👑 ', '❤️ ', '\U0001..."
3,4,[.\n.\n.\n.\n#staugustine #staugustinefl #stau...,[0],[4284913372],[[]],"[[., ., ., ., #, staugustine, #, staugustinefl...",['.\n.\n.\n.\n#staugustine #staugustinefl #sta...
4,5,[.\n.\n.\n.\n#boca #bocaraton #resortlife #exp...,"[0, 0, 0]","[4284913372, 534651664, 11230655630]","[[], [@patriciatorressl Waldorf?, @giles.ynwa ...","[[., ., ., ., #, boca, #, bocaraton, #, resort...",['.\n.\n.\n.\n#boca #bocaraton #resortlife #ex...


In [52]:
def string_to_list(df,columns):
    for col in columns:
        df[col]=df[col].apply(lambda x: ast.literal_eval(x))
        
for hotel in (hilton,hyatt,holiday,wyndham):
    string_to_list(hotel,["final"])

In [53]:
# 저장하는 함수 
def save_data(hyatt,hilton,holiday,wyndham):
    hyatt.to_csv("hyatt_comment",index=False)
    hilton.to_csv("hilton_comment",index=False)
    holiday.to_csv("holiday_comment",index=False)
    wyndham.to_csv("wyndham_comment",index=False)
save_data(hyatt,hilton,holiday,wyndham)

In [3]:
import pandas as pd 
def read_data():
    hyatt= pd.read_csv("hyatt_comment_final",index_col=[0]).reset_index()
    hilton=pd.read_csv("hilton_comment_final",index_col=[0]).reset_index()
    holiday=pd.read_csv("holiday_comment_final",index_col=[0]).reset_index()
    wyndham=pd.read_csv("wyndham_comment_final",index_col=[0]).reset_index()
    
    return (hilton,hyatt,holiday,wyndham)
hilton,hyatt,holiday,wyndham=read_data()
hilton.head()

,post_id,comment_token
0,1,"[['❤️❤️❤️', '💙🍪'], ['👏', 'we', ""'re"", 'pretty'..."
1,2,"[['This', 'is', 'awesome', '!', '!', '!', '👏👏👏..."
2,3,"[['Build', 'with', 'immortal', 'love', '👑'], [..."
3,4,"[['.', '.', '.', '.', '#', 'staugustine', '#',..."
4,5,"[['.', '.', '.', '.', '#', 'boca', '#', 'bocar..."
